### Параметры XGBRegressor (XGBRegressor Parameters)

Интерфейс обертки Scikit-Learn для XGBoost.

## Scikit-Learn API

### `class xgboost.XGBRegressor(**kwargs)`

- **`n_estimators (int)`**
    - Количество деревьев с градиентным усилением. Эквивалентно количеству раундов boost-инга (boosting rounds).


- **`max_depth (int) [default=6]`**
    - Максимальная глубина дерева. Увеличение этого значения сделает модель более сложной и с большей вероятностью переобучится (overfit).
    - Допустимые значения: $[0, \infty]$, `0` принимается только в lossguided для роста, когда tree_method установлен как hist и указывает отсутствие ограничения на глубину. Помните, что XGBoost агрессивно расходует память при обучении глубокого дерева.


- **`learning_rate (float) [default=0.3, alias: eta]`**
    - Уменьшение размера шага, используемое в обновлении, чтобы **предотвратить переобучение (overfitting)**. После каждого шага повышения мы можем напрямую получить веса новых функций, а eta уменьшает веса функций, чтобы сделать процесс повышения более консервативным.
    - Допустимые значения: $[0, 1]$


- `verbosity  (int) [default=1]`
    - Детальность печатных сообщений.
    - Допустимые значения:
        - 0 (silent)
        - 1 (warning)
        - 2 (info)
        - 3 (debug)
    - Иногда XGBoost пытается изменить конфигурации на основе эвристики, что отображается как предупреждающее сообщение. Если происходит неожиданное поведение, попробуйте увеличить значение подробности.
    - *Эвристический алгоритм (эвристика) — алгоритм решения задачи, включающий практический метод, не являющийся гарантированно точным или оптимальным, но достаточный для решения поставленной задачи. Позволяет ускорить решение задачи в тех случаях, когда точное решение не может быть найдено.


- `objective (string | callable)`
    - Устанавливает задачу обучения (learning task) и соответствующую цель обучения (learning objective) или настраиваемую целевую функцию (objective function), которая будет использоваться.
    - В этом случае objective parameter должен иметь сигнатуру `(y_true, y_pred) -> grad, hess`
        - `y_true: array_like of shape [n_samples]` - Целевые значения
        - `y_pred: array_like of shape [n_samples]` - Прогнозируемые значения
        - `grad: array_like of shape [n_samples]`- Значение градиента для каждой точки выборки.
        - `hess: array_like of shape [n_samples]` - Значение второй производной для каждой точки выборки


- `booster (string) [default='gbtree']`
    - Какой booster использовать.
    - Допустимые значения:
        - `gbtree`
        - `gblinear`
        - `dart`
    - `gbtree` и `dart` используют модели на основе дерева (tree based models), в то время как `gblinear` использует линейные функции (linear functions).


- `tree_method (string) [default='auto']`
    - Алгоритм построения дерева, используемый в XGBoost.
    - XGBoost поддерживает `approx`, `hist` и `gpu_hist` для распределенного обучения. Экспериментальная поддержка внешней памяти доступна для `approx` и `gpu_hist`.
    - Допустимые значения: `auto`, `exact`, `approx`, `hist`, `gpu_hist`, это комбинация часто используемых средств обновления. Для других средств обновления, таких как `refresh`, установите параметр `updater` напрямую.
        - `auto`: Использует эвристику, чтобы выбрать самый быстрый метод.
            - Для большего набора данных будет выбран `approx`. Рекомендуется попробовать `hist` и `gpu_hist` для повышения производительности с большим набором данных. `gpu_hist` поддерживает внешнюю память.
            - Поскольку старое поведение всегда использует точную жадность ([exact greedy](https://medium.com/analytics-vidhya/boosting-models-unwrapping-the-basic-exact-greedy-algorithm-b67d88c7189a)) на одной машине, пользователь получит сообщение, когда будет выбран `approx` алгоритм для уведомления об этом выборе.
        - `exact`: Точный жадный алгоритм ([exact greedy](https://medium.com/analytics-vidhya/boosting-models-unwrapping-the-basic-exact-greedy-algorithm-b67d88c7189a)). Перечисляет всех отдельных кандидатов.
        - `approx`: Приблизительный жадный алгоритм ([approximate greedy](https://towardsdatascience.com/why-xgboost-is-so-effective-3a193951e289)) с использованием квантилей ([quantile sketch](https://datasketches.apache.org/docs/Quantiles/QuantilesOverview.html)) и гистограммы градиента.
        - `hist`: Более быстрый приближенный жадный алгоритм Приблизительный жадный алгоритм ([approximate greedy](https://towardsdatascience.com/why-xgboost-is-so-effective-3a193951e289)), оптимизированный для гистограммы.
        - `gpu_hist`: Реализация алгоритма `hist` на GPU.

- `n_jobs (int)` 
    - Количество параллельных потоков, используемых для запуска `xgboost`.
    - При использовании с другими алгоритмами Scikit-Learn, такими как `grid search`, можно выбрать, какой алгоритм распараллеливать и балансировать потоки.
    - Создание конкуренции потоков ([thread contention](https://stackoverflow.com/questions/1970345/what-is-thread-contention#:~:text=Essentially%20thread%20contention%20is%20a,has%20unlocked%20that%20particular%20object.)) значительно замедлит работу обоих алгоритмов.

- `gamma (float) [default=0, alias: min_split_loss]`
    - Минимальное изменение значения функции потерь (Minimum loss reduction), необходимое для дальнейшего разбиения листового узла (leaf node) дерева (tree), то есть разделение листа на поддеревья. Чем больше `gamma`, тем более "консервативным" будет алгоритм.
    - Допустимые значения: $[0, \infty]$

- `min_child_weight (float) [default=1]`
    - Минимальная сумма веса экземпляра (hessian), необходимая для дочернего узла (child). Если на этапе разбиения дерева получается листовой узел (leaf node) с суммой веса экземпляра меньше min_child_weight, то процесс построения откажется от дальнейшего разделения.
    - В задаче линейной регрессии (linear regression) это просто соответствует минимальному количеству экземпляров (instances), которое должно быть в каждом узле. Чем больше min_child_weight, тем более "консервативным: будет алгоритм.
    - Допустимые значения: $[0, \infty]$
    

- `max_delta_step (int) [default=0]`
    - Максимальный шаг дельты, который допускается на выходе каждого листа (leaf output) для оценки веса каждого дерева.
    - Если значение установлено на `0`, это означает, что ограничения нет.
    - Если для него установлено `положительное значение`, это может помочь сделать шаг обновления более "консервативным".
    - *Обычно этот параметр не нужен*, но он может помочь в логистической регрессии (logistic regression), когда класс крайне несбалансирован (imbalanced).
    - Установка значения 1-10 может помочь контролировать обновление.
    - Допустимые значения: $[0, 1]$    
    
    
- **`subsample (float) [default=1]`**
    - Соотношение подвыборки обучающих примеров. Установка его на `0.5` означает, что XGBoost будет случайным образом выбирать половину обучающих данных перед выращиванием деревьев. И это *предотвратит переобучение (overfitting)*.
    - Подвыборка (subsample) будет выполняться один раз на каждой итерации boosting'а.
    - Допустимые значения: $(0, 1]$
    
- `colsample_bytree, colsample_bylevel, colsample_bynode (float)[default=1]`    
    - Это семейство параметров для подвыборки столбцов.
    - Все параметры `colsample_by*` имеют диапазон `(0, 1]`, значение по умолчанию 1, и определяют долю столбцов, подлежащих подвыборке.
    - `colsample_bytree` - это соотношение столбцов подвыборки при построении каждого дерева. Подвыборка (subsampling) выполняется один раз для каждого построенного дерева.
    - `colsample_bylevel` - отношение подвыборки столбцов для каждого уровня. Подвыборка (subsampling) выполняется один раз для каждого нового уровня глубины, достигнутого в дереве. Столбцы выбираются из набора столбцов, выбранных для *текущего дерева*.
    - `colsample_bynode` - это отношение подвыборки столбцов для каждого узла (разбиение). Подвыборка (subsampling) происходит каждый раз при оценке нового разделения. Столбцы выбираются из набора столбцов, выбранных для *текущего уровня*.
    - `colsample_by*` параметры работают [кумулятивно (накопительно)](https://ru.wiktionary.org/wiki/%D0%BA%D1%83%D0%BC%D1%83%D0%BB%D1%8F%D1%82%D0%B8%D0%B2%D0%BD%D1%8B%D0%B9). Например, комбинация `{'colsample_bytree':0.5, 'colsample_bylevel':0.5, 'colsample_bynode':0.5}` с 64 признаками оставит 8 признаков на выбор в каждом сплите (split).
    - В интерфейсе Python при использовании метода `hist`, `gpu_hist` или `exact` методов дерева (tree method) можно установить `feature_weights` для DMatrix, чтобы определить вероятность выбора каждой функции при использовании выборки столбцов (column sampling). Аналогичный параметр для метода `fit` есть в интерфейсе sklearn.
    
- `reg_alpha (float) [default=0, alias: alpha]`
    - Элемент регуляризации L1 ([lasso regression](http://www.machinelearning.ru/wiki/index.php?title=%D0%9B%D0%B0%D1%81%D1%81%D0%BE)) на весах.
    - Увеличение этого значения сделает модель более "консервативной".    
    
- `reg_lambda (float) [default=1, alias: lambda]`
    - Условие L2 регуляризации ([ridge regression или Tikhonov regularization](http://www.machinelearning.ru/wiki/index.php?title=%D0%A0%D0%B8%D0%B4%D0%B6-%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F)) весов.
    - Увеличение этого значения сделает модель более "консервативной".

- `scale_pos_weight (float) [default=1]`
    - Контролирует баланс положительных и отрицательных весов, что полезно для несбалансированных классов.
    - Типичное значение, которое следует учитывать: `sum(negative instances) / sum(positive instances)`.
    
- `base_score (float) [default=0.5]`
    - Начальная оценка прогноза для всех экземляров, глобальный bias.
    - При достаточном количестве итераций изменение этого значения не будет иметь большого эффекта.

- `random_state (int)`
    - Начальное число (порождающий элемент) для генератора случайных чисел ([random seed](https://en.wikipedia.org/wiki/Random_seed)).
    - xgboost использует генератор случайных чисел (random generator) только для подвыборки (subsampling). В остальном поведение детерминировано.
    
- `missing (float) [default=np.nan]`
    - Значение, которое должны быть использвано в качестве пропущенных значений (missing value).
    
- `num_parallel_tree (int) [default=1] `
    - Количество параллельных деревьев (trees), построенных на каждой итерации.
    - Эта опция используется для поддержки boost'инга случайного леса (random forest).
    
    
- `monotone_constraints (str)`
    - Ограничение переменной монотонности.

- `interaction_constraints (str)`
    - Ограничение для взаимодействия, представляющие разрешенные взаимодействия.
    - Ограничения должны быть указаны в виде списка вложений, например `[[0, 1], [2, 3, 4]]`, где каждый внутренний список представляет собой группу индексов функций, которым разрешено взаимодействовать друг с другом.
    
- `importance_type (string) [default="gain"]`
    - Тип важности признака для свойства `feature_importances_`:.
    - Допустимые значения: `"gain"`, `"weight"`, `"cover"`, `"total_gain"` или `"total_cover"`.
    
    
- `**kwargs (dict, optional)`
    - `**kwargs` для объекта XGBoost Booster. Полную документацию по параметрам можно найти [здесь](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst).
    - `**kwargs` не поддерживается scikit-learn. Нет гарантирии, что параметры, переданные через этот аргумент, будут правильно взаимодействовать с scikit-learn.

### `fit(X, y, *, sample_weight=None, base_margin=None, eval_set=None, eval_metric=None, early_stopping_rounds=None, verbose=True, xgb_model=None, sample_weight_eval_set=None, feature_weights=None, callbacks=None)`

Многократный вызов `fit()` приведет к повторному обучению модели с нуля (re-fit from scratch).

Чтобы возобновить обучение с предыдущей контрольной точки, явно передайте аргумент xgb_model.

- `X (array_like)` - Матрица признаков.
    
- `y (array_like)` - Labels ("правильные ответы").
    
- `sample_weight (array_like)` - Веса экземпляров.

- `base_margin (array_like)` - Глобальный байес (bias) для каждого экземпляра. 

- `eval_set (list, optional)` - Список пар кортежей (X, y) для использования в качестве проверочных наборов (validation sets), для которых будут вычисляться метрики. Показатели валидации (Validation metrics) помогают отслеживать производительность (performance) модели.

- `eval_metric (str, list of str, or callable, optional)` 
    - Если `str`, должна быть встроенная (built-in) метрика оценки для использования.
    - Если список `list of str`, должен быть список нескольких встроенных метрик оценки для использования.
    - Если вызываемая (`callable`),то пользовательская метрика оценки. Сигнатура вызова - `func(y_predicted, y_true)`, где `y_true` будет объектом `DMatrix`, может потребоваться вызвать метод `get_label`. Он должен возвращать пару `str, value`, где `str` - это имя для оценки, а `value` - это значение функции оценки. Вызываемая пользовательская задача (objective) всегда сводится к минимуму.
    
- **`early_stopping_rounds (int)`** 
    - Активирует раннюю остановку.
    - Чтобы продолжить обучение, показатель валидации должен улучшаться хотя бы раз в каждом раунде `early_stopping_rounds`.
    - Требуется хотя бы один элемент в `eval_set`.
    - Метод возвращает модель с последней итерации (а не самой лучшей).
    - Если в `eval_set` более одного элемента, для ранней остановки будет использоваться *последняя запись* (last entry).
    - Если в `eval_metric` более одной метрики, для ранней остановки будет использоваться *последняя метрика* (last metric).
    - Если произойдет ранняя остановка, модель будет иметь три дополнительных поля: `clf.best_score`, `clf.best_iteration` и `clf.best_ntree_limit`.
    
    

### `predict(data, output_margin=False, ntree_limit=None, validate_features=True, base_margin=None)`

**Эта функция не является потокобезопасной.**

Для каждого объекта-booster'а `predict` можно вызвать только из одного потока.

Если необходимо выполнить `predict` с использованием нескольких потоков, используйте `xgb.copy()`, чтобы сделать копии объекта модели, а затем вызовите `predict()`.

- `data (numpy.array/scipy.sparse)`
    - Данные для прогнозирования.
    
- `output_margin (bool)`
    - Выводить ли необработанное непреобразованное значение границы (margin).
    
- `ntree_limit (int)`
    - Ограничить количество деревьев в прогнозе
    - Значение по умолчанию: `best_ntree_limit`, если он определен (т.е. он был обучен с ранней остановкой (early stopping)), в противном случае - `0` (использовать все деревья).
    
- `validate_features (bool)`
    - Если это `True`, то будет произведена проверка (validate), что `feature_names` для `Booster` и `data` идентичны. В противном случае предполагается, что `feature_names` совпадают.

#### Глобальная конфигурация (**Global Configuration**)
- Параметр **`verbosity`** может быть установлен в глобальной области с помощью `xgb.config_context()`

    - Детальность печатных сообщений.

    - Допустимые значения:
        - 0 (silent)
        - 1 (warning)
        - 2 (info)
        - 3 (debug)

#### Общие параметры (**General parameters**)
относятся к тому, какой booster используется для boosting, обычно это древовидная (tree) или линейная (linear) модель.


- `booster [default= gbtree]`
    - Какой booster использовать.
    - Допустимые значения:
        - `gbtree`
        - `gblinear`
        - `dart`
    - `gbtree` и `dart` используют модели на основе дерева (tree based models), в то время как `gblinear` использует линейные функции (linear functions).


- `verbosity [default=1]`
    - Детальность печатных сообщений.
    - Допустимые значения:
        - 0 (silent)
        - 1 (warning)
        - 2 (info)
        - 3 (debug)
    - Иногда XGBoost пытается изменить конфигурации на основе эвристики, что отображается как предупреждающее сообщение. Если происходит неожиданное поведение, попробуйте увеличить значение подробности.
    - *Эвристический алгоритм (эвристика) — алгоритм решения задачи, включающий практический метод, не являющийся гарантированно точным или оптимальным, но достаточный для решения поставленной задачи. Позволяет ускорить решение задачи в тех случаях, когда точное решение не может быть найдено.


- `validate_parameters [default=True]`
    - Если установлено значение `True`, XGBoost будет выполнять проверку входных параметров, чтобы проверить, используется ли параметр или нет.
    - *Эта функция все еще экспериментальная. Ожидается несколько ложных срабатываний.*


- `nthread [default=максимальное количество доступных потоков, если не установлено]`
    - Количество параллельных потоков, используемых для запуска XGBoost.
    - *При выборе учитывайте конфликты потоков и гиперпоточность.*


- `disable_default_eval_metric [default='false']`
    - Используйте `'true'` (или `1`), чтобы отключить метрику по умолчанию.


- `num_pbuffer [устанавливается автоматически XGBoost, нет необходимости устанавливать пользователем]`
    - Размер буфера предсказания, обычно равный количеству обучающих экземпляров (training instances). Буферы используются для сохранения результатов прогноза последнего boosting шага.


- `num_feature [устанавливается автоматически XGBoost, нет необходимости устанавливать пользователем]`
    - Размер объекта, используемый при повышении, установлен на максимальный размер объекта.

#### Параметры booster'а (**Booster parameters**)
зависят от того какой booster выбран.

##### Параметры для Tree Booster (**Parameters for Tree Booster**)

- `eta [default=0.3, alias: learning_rate]`
    - Уменьшение размера шага, используемое в обновлении, чтобы **предотвратить переобучение (overfitting)**. После каждого шага повышения мы можем напрямую получить веса новых функций, а eta уменьшает веса функций, чтобы сделать процесс повышения более консервативным.
    - Допустимые значения: $[0, 1]$


- `gamma [default=0, alias: min_split_loss]`
    - Минимальное изменение значения функции потерь (Minimum loss reduction), необходимое для дальнейшего разбиения листового узла (leaf node) дерева (tree), то есть разделение листа на поддеревья. Чем больше `gamma`, тем более "консервативным" будет алгоритм.
    - Допустимые значения: $[0, \infty]$


- `max_depth [default=6]`
    - Максимальная глубина дерева. Увеличение этого значения сделает модель более сложной и с большей вероятностью переобучится (overfit).
    - Допустимые значения: $[0, \infty]$, `0` принимается только в lossguided для роста, когда tree_method установлен как hist и указывает отсутствие ограничения на глубину. Помните, что XGBoost агрессивно расходует память при обучении глубокого дерева.


- `min_child_weight [default=1]`
    - Минимальная сумма веса экземпляра (hessian), необходимая для дочернего узла (child). Если на этапе разбиения дерева получается листовой узел (leaf node) с суммой веса экземпляра меньше min_child_weight, то процесс построения откажется от дальнейшего разделения. В задаче линейной регрессии (linear regression) это просто соответствует минимальному количеству экземпляров (instances), которое должно быть в каждом узле. Чем больше min_child_weight, тем более "консервативным: будет алгоритм.
    - Допустимые значения: $[0, \infty]$


- `max_delta_step [default=0]`
    - Максимальный шаг дельты, который допускается на выходе каждого листа (leaf output).
    - Если значение установлено на `0`, это означает, что ограничения нет.
    - Если для него установлено `положительное значение`, это может помочь сделать шаг обновления более "консервативным".
    - *Обычно этот параметр не нужен*, но он может помочь в логистической регрессии (logistic regression), когда класс крайне несбалансирован (imbalanced).
    - Установка значения 1-10 может помочь контролировать обновление.
    - Допустимые значения: $[0, 1]$


- `subsample [default=1]`
    - Соотношение подвыборки обучающих примеров. Установка его на 0.5 означает, что XGBoost будет случайным образом выбирать половину обучающих данных перед выращиванием деревьев. И это предотвратит переобучение (overfitting). Подвыборка (subsample) будет выполняться один раз на каждой итерации boosting'а.
    - Допустимые значения: $(0, 1]$


- `sampling_method [default= uniform]`
    - Метод, используемый для выборки обучающих экземпляров.
    - `uniform`: каждый обучающий экземпляр имеет равную вероятность быть выбранным. Обычно для хороших результатов задают подвыборку $\ge$ 0.5.
    - `gradient_based`: вероятность выбора для каждого обучающего примера пропорциональна регуляризованному абсолютному значению градиентов (более конкретно, $\sqrt{g^{2} + \lambda*h^{2}}$.
    - Подвыборка (subsample) может быть установлена на уровне `0.1` без потери точности модели.
    - Обратите внимание, что этот метод выборки поддерживается только тогда, когда `tree_method` имеет значение `gpu_hist`; другие древовидные методы поддерживают только `uniform` выборку.


- `colsample_bytree, colsample_bylevel, colsample_bynode [default=1]`


- `lambda [default=1, alias: reg_lambda]`
    - Условие L2 регуляризации (ridge regression или Tikhonov regularization) весов.
    - Увеличение этого значения сделает модель более "консервативной".


- `alpha [default=0, alias: reg_alpha]`
    - Элемент регуляризации L1 (lasso regression) на весах.
    - Увеличение этого значения сделает модель более "консервативной".


- `tree_method string [default= auto]`


- `sketch_eps [default=0.03]`


- `scale_pos_weight [default=1]`


- `updater [default= grow_colmaker,prune]`


- `refresh_leaf [default=1]`
    - Это параметр обновления (refresh updater).
    - Когда этот флаг равен `1`, обновляется статистика листьев (leafs) дерева и узлов (nodes) дерева.
    - Когда он равен `0`, обновляется только статистика по узлам (nodes).


- `process_type [default= default]`


- `grow_policy [default= depthwise]`
    - Управляет способом добавления новых узлов в дерево.
    - В настоящее время поддерживается, только если для `tree_method` установлено значение `hist`.
    - Допустимые значения: `depthwise`, `lossguide`
        - `depthwise`: разбивать в узлах, ближайших к корню.
        - `lossguide`: разделить в узлах с наибольшим изменением потерь (loss change).


- `max_leaves [default=0]`
    - Максимальное количество добавляемых узлов (nodes). Актуально, только если установлено `grow_policy = lossguide`.


- `max_bin, [default=256]`
    - Используется, только если `tree_method` имеет значение `hist`.
    - Максимальное количество отдельных bin'ов для непрерывных bucket параметров (features).
    - Увеличение этого числа улучшает оптимальность разбиения за счет увеличения времени вычислений.


- `predictor, [default='auto']`


- `num_parallel_tree, [default=1]`
    - Количество параллельных деревьев (trees), построенных на каждой итерации. Эта опция используется для поддержки boost'инга случайного леса (random forest).


- `monotone_constraints`
    - Ограничение переменной монотонности.


- `interaction_constraints`
    - Ограничения для взаимодействия, представляющие разрешенные взаимодействия.
    - Ограничения должны быть указаны в виде списка вложений, например `[[0, 1], [2, 3, 4]]`, где каждый внутренний список представляет собой группу индексов функций, которым разрешено взаимодействовать друг с другом.


- *Дополнительный параметр для метода дерева `hist` и `gpu_hist`*
    - `single_precision_histogram, [default='false']`
        - Использовать одинарную точность для построения гистограмм вместо двойной точности.


- *Дополнительный параметр для метода дерева `gpu_hist`*
    - `deterministic_histogram, [default='true']`
        - Построение гистограммы на GPU детерминировано.
        - Построение гистограммы не является детерминированным из-за неассоциативного аспекта суммирования с плавающей запятой.
        - Используется процедура предварительного округления, чтобы устранить проблему, которая может немного снизить точность.
        - Установите значение `false`, чтобы отключить его.


- *Дополнительные параметры для `Dart Booster` (`booster=dart`)*

    - `sample_type [default= uniform]`


    - `normalize_type [default= tree]`


    - `rate_drop [default=0.0]`


    - `one_drop [default=0]`


    - `skip_drop [default=0.0]`


- *Параметры для `Linear Booster` (`booster=gblinear`)*

    - `lambda [default=0, alias: reg_lambda]`


    - `alpha [default=0, alias: reg_alpha]`


    - `updater [default= shotgun]`


    - `feature_selector [default= cyclic]`


    - `top_k [default=0]`


- *Параметры для `Tweedie Booster` (`objective=reg:tweedie`)*

    - `tweedie_variance_power [default=1.5]`

#### Параметры учебной задачи (**Learning task parameters**)
определяют сценарий обучения. Например, задачи регрессии (regression) могут использовать разные параметры с задачами ранжирования (ranking).


#### Параметры командной строки (**Command line parameters*)
относятся к поведению CLI-версии XGBoost.